In [ ]:
# 과거 히트한 디저트들 (학습 데이터용)
past_hits = [
    '마카롱', '크로플', '티라미수', '바스크치즈케이크',
    '휘낭시에', '카눌레', '마리토쪼', '달고나커피'
]

# 현재 떠오르는 디저트들 (예측 대상)
emerging_desserts = [
    '크림브륄레', '크룽지', '시폰케이크', '판나코타',
    '몽블랑', '밀크티타르트', '크로아상큐브'
]

# 전통 디저트 (비교 대상)
traditional = [
    '케이크', '쿠키', '아이스크림', '도넛', '와플'
]

In [ ]:
from pytrends.request import TrendReq
import time

def test_single_keyword(keyword):
    """
    단일 키워드 테스트
    """
    timeframes = ['today 3-m']
    
    for tf in timeframes:
        try:
            print(f"\n테스트: '{keyword}' with timeframe='{tf}'")
            pytrends = TrendReq(hl='ko', tz=540)
            pytrends.build_payload([keyword], timeframe=tf, geo='KR')
            data = pytrends.interest_over_time()
            
            if data is not None and not data.empty:
                print(f"✓ 성공! 데이터 포인트: {len(data)}개")
                print(data.head())
                return tf, data
            else:
                print(f"⚠ 데이터 없음")
                
        except Exception as e:
            print(f"❌ 에러: {e}")
            
        time.sleep(3)
    
    return None, None

# 테스트 실행
working_timeframe, test_data = test_single_keyword('두바이')

if working_timeframe:
    print(f"\n✓ 작동하는 timeframe: {working_timeframe}")
else:
    print("\n❌ 모든 timeframe 실패")

In [ ]:
import os
import sys
import urllib.request
client_id = "CsPxLxypgN1YGiITcN8E"
client_secret = "KuUsldEbIH"
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2025-09-01\",\"endDate\":\"2025-12-31\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
네이버 데이터랩 검색어 트렌드 API 사용 예제
API 문서: https://developers.naver.com/docs/serviceapi/datalab/search/search.md
"""

import requests
import json
from datetime import datetime


class NaverDataLabAPI:
    """네이버 데이터랩 검색어 트렌드 API 클래스"""
    
    def __init__(self, client_id, client_secret):
        """
        초기화
        
        Args:
            client_id (str): 네이버 API 클라이언트 ID
            client_secret (str): 네이버 API 클라이언트 Secret
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.api_url = "https://openapi.naver.com/v1/datalab/search"
    
    def get_search_trend(self, start_date, end_date, time_unit, keyword_groups, device="", ages=None, gender=""):
        """
        검색어 트렌드 조회
        
        Args:
            start_date (str): 조회 기간 시작일 (YYYY-MM-DD 형식)
            end_date (str): 조회 기간 종료일 (YYYY-MM-DD 형식)
            time_unit (str): 구간 단위 ('date', 'week', 'month')
            keyword_groups (list): 검색어 그룹 리스트
                예: [
                    {"groupName": "한글", "keywords": ["한글", "korean"]},
                    {"groupName": "영어", "keywords": ["영어", "english"]}
                ]
            device (str, optional): 기기 ('pc', 'mo', '' for 전체)
            ages (list, optional): 연령대 리스트 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
            gender (str, optional): 성별 ('m', 'f', '' for 전체)
        
        Returns:
            dict: API 응답 결과
        """
        # 헤더 설정
        headers = {
            'X-Naver-Client-Id': self.client_id,
            'X-Naver-Client-Secret': self.client_secret,
            'Content-Type': 'application/json'
        }
        
        # 요청 바디 생성
        body = {
            "startDate": start_date,
            "endDate": end_date,
            "timeUnit": time_unit,
            "keywordGroups": keyword_groups
        }
        
        # 선택적 매개변수 추가
        if device:
            body["device"] = device
        if ages:
            body["ages"] = ages
        if gender:
            body["gender"] = gender
        
        try:
            # API 요청
            response = requests.post(
                self.api_url,
                headers=headers,
                data=json.dumps(body)
            )
            
            # 응답 확인
            response.raise_for_status()
            
            return response.json()
            
        except requests.exceptions.HTTPError as e:
            print(f"HTTP 오류 발생: {e}")
            print(f"응답 내용: {response.text}")
            return None
        except Exception as e:
            print(f"오류 발생: {e}")
            return None
    
    def print_results(self, result):
        """
        결과를 보기 좋게 출력
        
        Args:
            result (dict): API 응답 결과
        """
        if not result:
            print("결과가 없습니다.")
            return
        
        print(f"\n{'='*80}")
        print(f"검색 기간: {result['startDate']} ~ {result['endDate']}")
        print(f"구간 단위: {result['timeUnit']}")
        print(f"{'='*80}\n")
        
        for idx, group_result in enumerate(result['results'], 1):
            print(f"[그룹 {idx}] {group_result['title']}")
            print(f"키워드: {', '.join(group_result['keywords'])}")
            print(f"\n{'날짜':<15} {'검색량 비율':<15}")
            print(f"{'-'*30}")
            
            for data_point in group_result['data']:
                print(f"{data_point['period']:<15} {data_point['ratio']:<15}")
            
            print(f"\n{'='*80}\n")


def main():
    """메인 함수 - 사용 예제"""
    
    # API 인증 정보 (여기에 실제 값을 입력하세요)
    CLIENT_ID = "CsPxLxypgN1YGiITcN8E"
    CLIENT_SECRET = "KuUsldEbIH"
    
    # API 인스턴스 생성
    api = NaverDataLabAPI(CLIENT_ID, CLIENT_SECRET)
    
    # 예제 1: 기본 검색어 트렌드 조회
    print("\n[예제 1] 기본 검색어 트렌드 조회")
    keyword_groups = [
        {
            "groupName": "디저트",
            "keywords": ["두바이 쫀득 쿠키", "두쫀쿠"]
        },
        {
            "groupName": "디저트2",
            "keywords": ["소금빵"]
        }
    ]
    
    result = api.get_search_trend(
        start_date="2025-01-01",
        end_date="2025-12-31",
        time_unit="month",
        keyword_groups=keyword_groups
    )
    
    if result:
        api.print_results(result)
    
    # 예제 2: 모바일 기기, 특정 연령대와 성별로 필터링
    print("\n[예제 2] 모바일, 20-30대 여성 대상 검색")
    keyword_groups_2 = [
        {
            "groupName": "화장품",
            "keywords": ["화장품", "cosmetics"]
        }
    ]
    
    result2 = api.get_search_trend(
        start_date="2025-01-01",
        end_date="2025-12-31",
        time_unit="month",
        keyword_groups=keyword_groups_2,
        device="mo",  # 모바일
        ages=["3", "4", "5"],  # 20-30대
        gender="f"  # 여성
    )
    
    if result2:
        api.print_results(result2)


if __name__ == "__main__":
    main()

In [ ]:
import requests
import pandas as pd

class KakaoMapAPI:
    def __init__(self, rest_api_key):
        """
        카카오 REST API 키 발급 방법:
        1. https://developers.kakao.com/ 접속
        2. 내 애플리케이션 > 앱 만들기
        3. 앱 설정 > 앱 키 > REST API 키 복사
        """
        self.api_key = rest_api_key
        self.base_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        
    def search_keyword(self, query, x=None, y=None, radius=None, page=1, size=15):
        """
        키워드로 장소 검색
        
        Parameters:
        - query: 검색 키워드 (예: '이태원 맛집')
        - x: 중심 좌표의 경도 (선택)
        - y: 중심 좌표의 위도 (선택)
        - radius: 검색 반경(m), 0~20000 (선택)
        - page: 페이지 번호 (1~45)
        - size: 한 페이지에 보여질 문서 수 (1~15)
        """
        headers = {'Authorization': f'KakaoAK {self.api_key}'}
        
        params = {
            'query': query,
            'page': page,
            'size': size
        }
        
        # 선택적 파라미터 추가
        if x and y:
            params['x'] = x
            params['y'] = y
        if radius:
            params['radius'] = radius
            
        response = requests.get(self.base_url, headers=headers, params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return None
    
    def get_all_results(self, query, x=None, y=None, radius=None, max_pages=3):
        """
        여러 페이지의 검색 결과를 모두 가져오기
        """
        all_places = []
        
        for page in range(1, max_pages + 1):
            result = self.search_keyword(query, x, y, radius, page=page)
            
            if result and result['documents']:
                all_places.extend(result['documents'])
                
                # 마지막 페이지면 중단
                if result['meta']['is_end']:
                    break
            else:
                break
                
        return all_places
    
    def display_results(self, places):
        """
        검색 결과를 보기 좋게 출력
        """
        if not places:
            print("검색 결과가 없습니다.")
            return
            
        print(f"\n총 {len(places)}개의 장소를 찾았습니다.\n")
        print("=" * 80)
        
        for i, place in enumerate(places, 1):
            print(f"\n[{i}] {place['place_name']}")
            print(f"    카테고리: {place['category_name']}")
            print(f"    주소: {place['address_name']}")
            print(f"    도로명: {place.get('road_address_name', 'N/A')}")
            print(f"    전화번호: {place.get('phone', 'N/A')}")
            print(f"    위도/경도: {place['y']}, {place['x']}")
            print(f"    카카오맵 URL: {place['place_url']}")
            print("-" * 80)
    
    def to_dataframe(self, places):
        """
        검색 결과를 DataFrame으로 변환
        """
        if not places:
            return pd.DataFrame()
            
        df = pd.DataFrame(places)
        
        # 필요한 컬럼만 선택
        columns = ['place_name', 'category_name', 'address_name', 
                   'road_address_name', 'phone', 'x', 'y', 'place_url']
        
        return df[columns]


# 사용 예시
if __name__ == "__main__":
    # API 키 설정 (본인의 REST API 키로 교체)
    API_KEY = 'd466301e9bc71ca31a3a0e9a985f8d07'
    
    # API 객체 생성
    kakao = KakaoMapAPI(API_KEY)
    
    # 1. 기본 검색 (이태원 맛집)
    print("=" * 80)
    print("검색어: 이태원 맛집")
    print("=" * 80)
    
    places = kakao.get_all_results('소금빵', max_pages=100)
    kakao.display_results(places)
    
    # 2. 특정 위치 중심으로 검색 (이태원역 기준 1km 반경)
    print("\n\n" + "=" * 80)
    print("검색어: 카페 (이태원역 1km 반경)")
    print("=" * 80)
    
    places2 = kakao.get_all_results(
        query='카페',
        x='126.9944',  # 이태원역 경도
        y='37.5345',   # 이태원역 위도
        radius=1000,   # 1km
        max_pages=1
    )
    kakao.display_results(places2)
    
    # 3. DataFrame으로 변환하여 저장
    df = kakao.to_dataframe(places)
    print("\n\nDataFrame으로 변환:")
    print(df.head())
    
    # CSV로 저장
    df.to_csv('이태원_맛집.csv', index=False, encoding='utf-8-sig')
    print("\n결과를 '이태원_맛집.csv'에 저장했습니다.")

In [12]:
# 동 이름으로 좌표 검색
def get_dong_coordinates(kakao, dong_name):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {'Authorization': f'KakaoAK {kakao.api_key}'}
    params = {'query': dong_name}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['documents']:
            doc = data['documents'][0]
            return float(doc['x']), float(doc['y'])
    return None, None

# 사용
x, y = get_dong_coordinates(kakao, '이태원동')
print(f"이태원동 좌표: {x}, {y}")

이태원동 좌표: 126.991527297365, 37.5402662116134


In [ ]:
import requests
import pandas as pd
import time
from collections import defaultdict

class KakaoMapAPI:
    def __init__(self, rest_api_key):
        self.api_key = rest_api_key
        self.base_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
        
    def search_keyword(self, query, x=None, y=None, radius=None, page=1, size=15):
        headers = {'Authorization': f'KakaoAK {self.api_key}'}
        
        params = {
            'query': query,
            'page': page,
            'size': size
        }
        
        if x and y:
            params['x'] = x
            params['y'] = y
        if radius:
            params['radius'] = radius
            
        response = requests.get(self.base_url, headers=headers, params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    
    def search_by_area(self, keyword, area_name):
        """
        특정 지역(구, 동, 로)에서 키워드 검색
        
        Parameters:
        - keyword: 검색 키워드 (예: '두쫀쿠')
        - area_name: 지역명 (예: '용산구', '이태원로', '경리단길')
        """
        query = f"{area_name} {keyword}"
        
        all_places = []
        seen_ids = set()
        
        for page in range(1, 4):  # 최대 45개
            result = self.search_keyword(query, page=page, size=15)
            
            if result and result['documents']:
                for place in result['documents']:
                    place_id = place['id']
                    
                    if place_id not in seen_ids:
                        seen_ids.add(place_id)
                        all_places.append(place)
                
                if result['meta']['is_end']:
                    break
            else:
                break
            
            time.sleep(0.1)
        
        return all_places
    
    def search_seoul_all(self, keyword):
        """
        서울시 전체 검색 후 도로명별로 분류
        """
        print(f"🔍 서울시 전체에서 '{keyword}' 검색 중...")
        
        query = f"서울 {keyword}"
        all_places = []
        seen_ids = set()
        
        # 서울시 전체 검색 (최대 45개)
        for page in range(1, 4):
            result = self.search_keyword(query, page=page, size=ㅠㅠ)
            
            if result and result['documents']:
                for place in result['documents']:
                    place_id = place['id']
                    
                    if place_id not in seen_ids:
                        # 서울시인지 확인
                        if '서울' in place['address_name']:
                            seen_ids.add(place_id)
                            all_places.append(place)
                
                if result['meta']['is_end']:
                    break
            else:
                break
            
            time.sleep(0.1)
        
        print(f"   ✅ 총 {len(all_places)}개 발견")
        
        # 도로명별로 분류
        road_dict = defaultdict(list)
        
        for place in all_places:
            road_name = place.get('road_address_name', '')
            
            if road_name:
                # 도로명 추출 (예: "서울 용산구 이태원로 123" → "이태원로")
                parts = road_name.split()
                if len(parts) >= 3:
                    road = parts[2]  # "이태원로"
                    road_dict[road].append(place)
            else:
                road_dict['도로명 없음'].append(place)
        
        return all_places, road_dict
    
    def search_multiple_roads(self, keyword, road_list):
        """
        여러 도로명에서 검색
        
        Parameters:
        - keyword: 검색 키워드
        - road_list: ['이태원로', '경리단길', '한남대로', ...]
        """
        results = {}
        
        for road_name in road_list:
            print(f"\n🔍 {road_name} 검색 중...")
            places = self.search_by_area(keyword, road_name)
            
            # 실제로 해당 도로에 있는지 필터링
            filtered_places = []
            for place in places:
                road_addr = place.get('road_address_name', '')
                if road_name in road_addr:
                    filtered_places.append(place)
            
            results[road_name] = filtered_places
            print(f"   ✅ {road_name}: {len(filtered_places)}개 발견")
            
            time.sleep(0.3)
        
        return results
    
    def summarize_by_road(self, results):
        """
        도로명별 검색 결과 요약
        """
        summary = []
        
        for road_name, places in sorted(results.items(), key=lambda x: len(x[1]), reverse=True):
            if places:
                summary.append({
                    '도로명': road_name,
                    '매장수': len(places),
                    '매장명': ', '.join([p['place_name'] for p in places[:3]]) + 
                             (f' 외 {len(places)-3}개' if len(places) > 3 else '')
                })
        
        df = pd.DataFrame(summary)
        return df


# ==================== 사용 예시 ====================

API_KEY = 'd466301e9bc71ca31a3a0e9a985f8d07'
kakao = KakaoMapAPI(API_KEY)

# ========================================
# 방법 1: 서울시 전체 검색 → 자동으로 도로명별 분류
# ========================================
print("\n" + "="*80)
print("방법 1: 서울시 전체 검색")
print("="*80)

all_places, road_dict = kakao.search_seoul_all('카페')

# 도로명별 통계
print("\n📊 도로명별 매장 수:")
print("-"*80)
for road, places in sorted(road_dict.items(), key=lambda x: len(x[1]), reverse=True):
    if places:
        print(f"{road}: {len(places)}개")
        for place in places:
            print(f"  - {place['place_name']} ({place['road_address_name']})")

# ========================================
# 방법 2: 특정 도로명 리스트로 검색 (더 정확)
# ========================================
print("\n" + "="*80)
print("방법 2: 특정 도로명으로 검색")
print("="*80)

# 서울 주요 도로명 리스트
seoul_roads = [
    # 용산구
    '이태원로',
    '이태원로55길',
    '경리단길',
    '한남대로',
    '녹사평대로',
    
    # 강남구
    '테헤란로',
    '강남대로',
    '논현로',
    '압구정로',
    '도산대로',
    '선릉로',
    '봉은사로',
    
    # 서초구
    '서초대로',
    '반포대로',
    
    # 마포구
    '와우산로',
    '홍익로',
    '양화로',
    '독막로',
    
    # 종로구
    '삼청로',
    '북촌로',
    '율곡로',
    '종로',
    
    # 중구
    '명동길',
    '을지로',
    '충무로',
    
    # 송파구
    '올림픽로',
    '송파대로',
    
    # 성동구
    '왕십리로',
    '성수이로',
    
    # 영등포구
    '여의대로',
    '국회대로',
]

results = kakao.search_multiple_roads('두쫀쿠', seoul_roads)

# 결과 요약
df_summary = kakao.summarize_by_road(results)
print("\n" + "="*80)
print("📊 도로명별 두쫀쿠 매장 현황")
print("="*80)
print(df_summary.to_string(index=False))

# 상세 결과
print("\n" + "="*80)
print("📍 전체 매장 목록")
print("="*80)

for road_name, places in sorted(results.items(), key=lambda x: len(x[1]), reverse=True):
    if places:
        print(f"\n【{road_name}】 ({len(places)}개)")
        for i, place in enumerate(places, 1):
            print(f"  {i}. {place['place_name']}")
            print(f"     도로명: {place.get('road_address_name', 'N/A')}")
            print(f"     지번: {place['address_name']}")
            print(f"     전화: {place.get('phone', 'N/A')}")

# CSV 저장
all_data = []
for road_name, places in results.items():
    for place in places:
        all_data.append({
            '도로명': road_name,
            '매장명': place['place_name'],
            '도로명주소': place.get('road_address_name', ''),
            '지번주소': place['address_name'],
            '전화번호': place.get('phone', ''),
            '카테고리': place['category_name'],
            '위도': place['y'],
            '경도': place['x'],
            '카카오맵URL': place['place_url']
        })

df_all = pd.DataFrame(all_data)
df_all.to_csv('도로명별_두쫀쿠_매장.csv', index=False, encoding='utf-8-sig')
print("\n💾 결과를 '도로명별_두쫀쿠_매장.csv'에 저장했습니다.")

# 통계 출력
print("\n" + "="*80)
print("📈 통계")
print("="*80)
print(f"검색한 도로명 수: {len(seoul_roads)}개")
print(f"매장이 있는 도로명: {len([r for r in results.values() if r])}개")
print(f"총 매장 수: {len(all_data)}개")
print(f"매장이 가장 많은 도로: {max(results.items(), key=lambda x: len(x[1]))[0]} ({max(len(p) for p in results.values())}개)")


방법 1: 서울시 전체 검색
🔍 서울시 전체에서 '카페' 검색 중...
   ✅ 총 45개 발견

📊 도로명별 매장 수:
--------------------------------------------------------------------------------
상계로: 2개
  - 감각 (서울 노원구 상계로 77)
  - 시드누아 노원본점 (서울 노원구 상계로 84)
테헤란로6길: 1개
  - 키이스케이프 강남 더오름 (서울 강남구 테헤란로6길 30)
율곡로: 1개
  - 프릳츠 원서점 (서울 종로구 율곡로 83)
선릉로158길: 1개
  - 고센인코퍼레이티드 (서울 강남구 선릉로158길 18)
삼일대로17길: 1개
  - 반쥴 (서울 종로구 삼일대로17길 23)
성미산로23길: 1개
  - 땡스오트 연남 (서울 마포구 성미산로23길 68)
연무장7가길: 1개
  - 베통 성수 (서울 성동구 연무장7가길 8)
압구정로2길: 1개
  - 나따오비까 신사점 (서울 강남구 압구정로2길 37)
역삼로3길: 1개
  - 쉼스토리 (서울 강남구 역삼로3길 17)
성지1길: 1개
  - 디벙크 (서울 마포구 성지1길 30)
남부순환로: 1개
  - 테라로사커피 예술의전당점 (서울 서초구 남부순환로 2406)
경인로: 1개
  - 영등포 카페드람브르 (서울 영등포구 경인로 867)
강남대로84길: 1개
  - 비트포비아 강남던전점 (서울 강남구 강남대로84길 33)
연서로29길: 1개
  - YM커피프로젝트 (서울 은평구 연서로29길 21-8)
월드컵로12길: 1개
  - 앤트러사이트 서교점 (서울 마포구 월드컵로12길 11)
정동길: 1개
  - 라운드앤드 (서울 중구 정동길 35)
성미산로: 1개
  - 네시사분 (서울 마포구 성미산로 190-8)
양화로16길: 1개
  - 943킹스크로스 (서울 마포구 양화로16길 24)
디지털로32나길: 1개
  - 메이비 (서울 구로구 디지털로32나길 51)
도로명 없음: 1개
  - 뷰376카페 ()
동작대로1길: 1개
 

In [24]:
pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------ --------------------------------- 1.6/9.7 MB 11.8 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.7 MB 11.8 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.7 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.7 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 11.6 MB/s  0:00:00

  Attempting uninstall: urllib3

    Found existing installation: urllib3 1.26.15

    Uninstalling urllib3-1.26.15:

      Successfully uninstalled urllib3-1.26.15

   ----- ---------------------------------- 1/8 [urllib3]
   ----- ---------------------------------- 1/8 [urllib3]
   ----- ---------------------------------- 1/8 [urllib3]
   ---------- ----------------------------- 2/8 [sniffio]
   ------------------------- -------------- 5/8 [trio]
   ------------------------- -------------- 5/8 [trio]
   ------------------------- -----------

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def crawl_naver_map(keyword, area):
    """
    네이버 지도에서 직접 크롤링
    """
    driver = webdriver.Chrome()
    
    # 네이버 지도 접속
    driver.get('https://map.naver.com/')
    time.sleep(2)
    
    # 검색
    search_box = driver.find_element(By.CLASS_NAME, 'input_search')
    search_box.send_keys(f'{area} {keyword}')
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)
    
    # 결과 파싱
    places = []
    
    # iframe 전환
    driver.switch_to.frame('searchIframe')
    
    # 매장 리스트 가져오기
    items = driver.find_elements(By.CLASS_NAME, 'place_bluelink')
    
    for item in items[:100]:  # 최대 100개
        try:
            name = item.text
            item.click()
            time.sleep(1)
            
            # 상세 정보 iframe으로 전환
            driver.switch_to.default_content()
            driver.switch_to.frame('entryIframe')
            
            # 주소 가져오기
            address = driver.find_element(By.CLASS_NAME, 'addr').text
            
            places.append({
                '매장명': name,
                '주소': address
            })
            
            # 다시 검색 결과로 돌아가기
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')
            
        except Exception as e:
            print(f"Error: {e}")
            continue
    
    driver.quit()
    return places

# 사용
places = crawl_naver_map('두쫀쿠', '서울 용산구')

Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":".addr"}
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6728688d5
	0x7ff672868930
	0x7ff67264165d
	0x7ff672699a33
	0x7ff672699d3c
	0x7ff6726edf67
	0x7ff6726eac97
	0x7ff67268ac29
	0x7ff67268ba93
	0x7ff672b80620
	0x7ff672b7af60
	0x7ff672b996c6
	0x7ff672885dd4
	0x7ff67288ed7c
	0x7ff672871ff4
	0x7ff6728721a5
	0x7ff672857ed2
	0x7ffa1db1e8d7
	0x7ffa1f31bf6c

Error: Message: no such element: element not found
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6728688d5
	0x7ff672868930
	0x7ff672641

In [33]:
# ==================== 수정된 전체 코드 ====================
# ==================== 수정된 전체 코드 ====================

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def crawl_naver_map_fixed(keyword, area):
    """
    네이버 지도 크롤링 (오류 수정 버전)
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    
    driver = webdriver.Chrome(options=options)
    places = []
    
    try:
        driver.get('https://map.naver.com/')
        time.sleep(2)
        
        search_query = f'{area} {keyword}'
        print(f"🔍 검색어: {search_query}")
        
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input.input_search'))
        )
        search_box.click()
        search_box.clear()
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.ENTER)
        
        time.sleep(3)
        
        driver.switch_to.frame('searchIframe')
        time.sleep(2)
        
        # 스크롤
        try:
            scroll_area = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
            last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_area)
            
            for _ in range(5):
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_area)
                time.sleep(1)
                new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_area)
                if new_height == last_height:
                    break
                last_height = new_height
        except:
            print("스크롤 영역을 찾을 수 없습니다.")
        
        # 매장 리스트 가져오기
        items = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
        
        if not items:
            # 다른 셀렉터 시도
            items = driver.find_elements(By.CSS_SELECTOR, 'li[role="listitem"]')
        
        print(f"✅ {len(items)}개 항목 발견")
        
        for idx, item in enumerate(items[:50], 1):
            try:
                # 매장명 (여러 셀렉터 시도)
                name = ''
                try:
                    name = item.find_element(By.CSS_SELECTOR, 'span.TYaxT').text
                except:
                    try:
                        name = item.find_element(By.CSS_SELECTOR, 'span.place_bluelink').text
                    except:
                        name = '이름 없음'
                
                # 주소
                address = ''
                try:
                    address = item.find_element(By.CSS_SELECTOR, 'span.LDgIH').text
                except:
                    try:
                        address = item.find_element(By.CSS_SELECTOR, 'div.addr').text
                    except:
                        address = ''
                
                # 카테고리
                category = ''
                try:
                    category = item.find_element(By.CSS_SELECTOR, 'span.KCMnt').text
                except:
                    pass
                
                # 전화번호
                phone = ''
                try:
                    phone = item.find_element(By.CSS_SELECTOR, 'span.dry06').text
                except:
                    pass
                
                if name and name != '이름 없음':
                    places.append({
                        '매장명': name,
                        '카테고리': category,
                        '주소': address,
                        '전화번호': phone
                    })
                    
                    print(f"  [{idx}] {name}")
                    if address:
                        print(f"       {address}")
                
            except Exception as e:
                print(f"  항목 {idx} 파싱 실패: {e}")
                continue
        
    except Exception as e:
        print(f"❌ 크롤링 오류: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        driver.quit()
    
    return places


# ==================== 실행 ====================

places = crawl_naver_map_fixed('두쫀쿠', '서울')

print(f"\n총 {len(places)}개 매장 수집 완료")

# DataFrame 생성 전에 데이터 확인
if not places:
    print("❌ 수집된 데이터가 없습니다!")
    print("네이버 지도 HTML 구조가 변경되었을 수 있습니다.")
    print("\n대신 네이버 검색 API 사용을 강력히 권장합니다!")
else:
    df = pd.DataFrame(places)
    
    # 컬럼 확인
    print("\n컬럼 목록:", df.columns.tolist())
    print("\n데이터 샘플:")
    print(df.head())
    
    # 도로명 추출 함수
    def extract_road_name(address):
        if pd.isna(address) or not address:
            return '알 수 없음'
        
        parts = address.split()
        for part in parts:
            if '로' in part or '길' in part:
                return part
        return '알 수 없음'
    
    # '주소' 컬럼이 있는지 확인
    if '주소' in df.columns:
        df['도로명'] = df['주소'].apply(extract_road_name)
    else:
        print("❌ '주소' 컬럼이 없습니다!")
        print("사용 가능한 컬럼:", df.columns.tolist())
    
    # 도로명별 통계
    if '도로명' in df.columns:
        print("\n" + "="*80)
        print("📊 도로명별 두쫀쿠 매장 수")
        print("="*80)
        
        road_stats = df.groupby('도로명').size().sort_values(ascending=False)
        print(road_stats)
        
        # 상세 출력
        print("\n" + "="*80)
        print("📍 전체 매장 목록")
        print("="*80)
        
        for road in road_stats.index:
            if road != '알 수 없음':
                road_places = df[df['도로명'] == road]
                print(f"\n【{road}】 ({len(road_places)}개)")
                for idx, place in road_places.iterrows():
                    print(f"  - {place['매장명']}")
                    if place['주소']:
                        print(f"    {place['주소']}")
                    if place['전화번호']:
                        print(f"    {place['전화번호']}")
    
    # CSV 저장
    df.to_csv('네이버지도_두쫀쿠_매장.csv', index=False, encoding='utf-8-sig')
    print(f"\n💾 {len(df)}개 매장을 CSV로 저장했습니다.")


🔍 검색어: 서울 두쫀쿠
✅ 0개 항목 발견

총 0개 매장 수집 완료
❌ 수집된 데이터가 없습니다!
네이버 지도 HTML 구조가 변경되었을 수 있습니다.

대신 네이버 검색 API 사용을 강력히 권장합니다!


In [34]:
import requests
import pandas as pd
import time
from collections import defaultdict

class NaverLocalAPI:
    def __init__(self, client_id, client_secret):
        """
        네이버 지역 검색 API 클래스
        
        Parameters:
        - client_id: 네이버 개발자센터에서 발급받은 Client ID
        - client_secret: 네이버 개발자센터에서 발급받은 Client Secret
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = 'https://openapi.naver.com/v1/search/local.json'
    
    def search(self, query, display=100, start=1, sort='random'):
        """
        지역 검색
        
        Parameters:
        - query: 검색어 (예: '서울 용산구 두쫀쿠')
        - display: 한 번에 가져올 결과 수 (최대 100)
        - start: 검색 시작 위치 (1~100)
        - sort: 정렬 방식 ('random' 또는 'comment')
        
        Returns:
        - 검색 결과 리스트
        """
        headers = {
            'X-Naver-Client-Id': self.client_id,
            'X-Naver-Client-Secret': self.client_secret
        }
        
        params = {
            'query': query,
            'display': display,
            'start': start,
            'sort': sort
        }
        
        try:
            response = requests.get(self.base_url, headers=headers, params=params)
            
            if response.status_code == 200:
                data = response.json()
                return data['items']
            else:
                print(f"❌ API 오류: {response.status_code}")
                print(f"응답: {response.text}")
                return []
        
        except Exception as e:
            print(f"❌ 요청 실패: {e}")
            return []
    
    def search_multiple_areas(self, keyword, areas):
        """
        여러 지역에서 검색
        
        Parameters:
        - keyword: 검색 키워드 (예: '두쫀쿠')
        - areas: 지역 리스트 (예: ['서울 용산구', '서울 강남구'])
        
        Returns:
        - 전체 검색 결과 리스트
        """
        all_results = []
        seen_ids = set()  # 중복 제거용
        
        for area in areas:
            query = f"{area} {keyword}"
            print(f"🔍 검색 중: {query}")
            
            items = self.search(query)
            
            for item in items:
                # 중복 제거 (매장명 + 주소로 판단)
                item_id = (
                    item['title'].replace('<b>', '').replace('</b>', ''),
                    item['address']
                )
                
                if item_id not in seen_ids:
                    seen_ids.add(item_id)
                    all_results.append(item)
            
            print(f"   ✅ {len(items)}개 발견 (누적: {len(all_results)}개)")
            time.sleep(0.1)  # API 호출 제한 방지
        
        return all_results
    
    def clean_html_tags(self, text):
        """HTML 태그 제거"""
        return text.replace('<b>', '').replace('</b>', '')
    
    def to_dataframe(self, items):
        """
        검색 결과를 DataFrame으로 변환
        """
        if not items:
            return pd.DataFrame()
        
        data = []
        for item in items:
            data.append({
                '매장명': self.clean_html_tags(item['title']),
                '도로명주소': self.clean_html_tags(item.get('roadAddress', '')),
                '지번주소': self.clean_html_tags(item['address']),
                '카테고리': item.get('category', ''),
                '전화번호': item.get('telephone', ''),
                '위도': item.get('mapy', ''),  # 주의: y가 위도
                '경도': item.get('mapx', ''),  # 주의: x가 경도
                '네이버링크': item.get('link', '')
            })
        
        return pd.DataFrame(data)
    
    def extract_road_name(self, address):
        """
        도로명 주소에서 '~로', '~길' 추출
        
        예시:
        - '서울특별시 용산구 이태원로 123' → '이태원로'
        - '서울 강남구 테헤란로7길 22' → '테헤란로7길'
        """
        if not address or address == '':
            return '알 수 없음'
        
        parts = address.split()
        
        for part in parts:
            if '로' in part or '길' in part:
                return part
        
        return '알 수 없음'
    
    def analyze_by_road(self, df):
        """
        도로명별로 분석
        """
        if df.empty:
            return pd.DataFrame()
        
        # 도로명 추출
        df['도로명'] = df['도로명주소'].apply(self.extract_road_name)
        
        # 도로명별 통계
        road_stats = df.groupby('도로명').agg({
            '매장명': 'count',
            '도로명주소': lambda x: list(x)[:3]  # 샘플 3개
        }).rename(columns={'매장명': '매장수'})
        
        road_stats = road_stats.sort_values('매장수', ascending=False)
        
        return road_stats


# ==================== 실제 사용 예시 ====================

# 🔑 여기에 본인의 키를 입력하세요!
CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'

# API 객체 생성
naver = NaverLocalAPI(CLIENT_ID, CLIENT_SECRET)

# ==================== 방법 1: 단일 지역 검색 ====================
print("\n" + "="*80)
print("방법 1: 서울 용산구에서 두쫀쿠 검색")
print("="*80)

items = naver.search('서울 용산구 두쫀쿠')
print(f"\n총 {len(items)}개 발견")

# DataFrame 변환
df = naver.to_dataframe(items)
print("\n검색 결과:")
print(df[['매장명', '도로명주소', '전화번호']].to_string(index=False))

# ==================== 방법 2: 여러 지역 검색 ====================
print("\n" + "="*80)
print("방법 2: 서울 여러 구에서 두쫀쿠 검색")
print("="*80)

# 서울 주요 구
seoul_areas = [
    '서울 용산구',
    '서울 강남구',
    '서울 마포구',
    '서울 종로구',
    '서울 중구',
    '서울 서초구',
    '서울 송파구',
    '서울 성동구',
    '서울 영등포구',
    '서울 광진구'
]

all_items = naver.search_multiple_areas('두쫀쿠', seoul_areas)

print(f"\n총 {len(all_items)}개 매장 발견")

# DataFrame 변환
df_all = naver.to_dataframe(all_items)

# ==================== 도로명별 분석 ====================
print("\n" + "="*80)
print("📊 도로명별 두쫀쿠 매장 통계")
print("="*80)

road_stats = naver.analyze_by_road(df_all)
print(road_stats)

# 상위 10개 도로명 상세 출력
print("\n" + "="*80)
print("📍 매장이 가장 많은 도로 TOP 10")
print("="*80)

top_roads = road_stats.head(10).index

for road in top_roads:
    if road != '알 수 없음':
        road_places = df_all[df_all['도로명주소'].apply(naver.extract_road_name) == road]
        
        print(f"\n【{road}】 - {len(road_places)}개 매장")
        print("-" * 60)
        
        for idx, row in road_places.iterrows():
            print(f"  • {row['매장명']}")
            print(f"    {row['도로명주소']}")
            if row['전화번호']:
                print(f"    ☎ {row['전화번호']}")
            print()

# ==================== CSV 저장 ====================
df_all.to_csv('네이버API_두쫀쿠_전체.csv', index=False, encoding='utf-8-sig')
print(f"\n💾 전체 결과를 '네이버API_두쫀쿠_전체.csv'에 저장했습니다.")

# 도로명별 통계도 저장
road_stats.to_csv('네이버API_도로명별_통계.csv', encoding='utf-8-sig')
print(f"💾 도로명별 통계를 '네이버API_도로명별_통계.csv'에 저장했습니다.")

# ==================== 요약 출력 ====================
print("\n" + "="*80)
print("📈 최종 요약")
print("="*80)
print(f"검색한 구: {len(seoul_areas)}개")
print(f"총 매장 수: {len(df_all)}개")
print(f"검색된 도로명: {len(road_stats)}개")
print(f"가장 많은 매장이 있는 도로: {road_stats.index[0]} ({road_stats.iloc[0]['매장수']}개)")
print("="*80)


방법 1: 서울 용산구에서 두쫀쿠 검색
❌ API 오류: 401
응답: {"errorMessage":"NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)","errorCode":"024"}

총 0개 발견

검색 결과:


KeyError: "None of [Index(['매장명', '도로명주소', '전화번호'], dtype='object')] are in the [columns]"